데이터 로딩 및 전처리:

data.csv에서 데이터를 로드하고, 결측치가 포함된 행을 식별한다.

결측치가 아닌 데이터를 기준으로 모델을 훈련한다.

자동 인코더 모델 구축 및 훈련:

LSTM 레이어를 포함하는 인코더와 디코더를 구축한다.

모델을 훈련 데이터로 훈련한다.

결측치 보간:

모델을 사용하여 결측치가 있는 데이터 포인트를 보간한다.

보간된 값을 sample_submission.csv의 형식에 맞게 저장한다.

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 데이터 로드 및 준비
data = pd.read_csv('C:/Users/82106/Desktop/데이터 분석 프로젝트 2/dacon 결측치 보간/open/data.csv')

In [9]:
# 결측치가 있는 행과 없는 행을 분리
train_data = data.dropna()
missing_data = data[data.isnull().any(axis=1)]

In [10]:
# 데이터셋 분할
X_train, X_test = train_test_split(train_data['Value'].values, test_size=0.2, random_state=42)

In [11]:
# 데이터 형태 조정 (LSTM은 [samples, timesteps, features] 형태의 입력 필요)
X_train = X_train.reshape((X_train.shape[0], 1, 1))
X_test = X_test.reshape((X_test.shape[0], 1, 1))

In [13]:
#모델 구성
input_dim=X_train.shape[-1]
timesteps=X_train.shape[1]

inputs=Input(shape=(timesteps,input_dim))
encoded=LSTM(10)(inputs)

decoded=RepeatVector(timesteps)(encoded)
decoded=LSTM(input_dim,return_sequences=True)(decoded)

In [14]:
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

In [15]:
# 모델 훈련
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 585.4236 - val_loss: 570.9173
Epoch 2/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 566.7658 - val_loss: 557.2694
Epoch 3/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 557.2470 - val_loss: 553.1809
Epoch 4/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 553.1150 - val_loss: 552.5288
Epoch 5/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 552.9546 - val_loss: 552.3027
Epoch 6/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 552.2287 - val_loss: 552.1712
Epoch 7/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 554.7648 - val_loss: 552.0845
Epoch 8/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 553.9211 - val_loss: 552.0230
Epoch 9/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 552.7219 - val_loss: 551.9775
Epoch 10/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 555.8276 - val_loss: 551.9426
Epoch 11/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 553.8932 - val_loss: 551.9152
Epoch 1

In [23]:
# 결측치 보간
missing_data_filled = missing_data.copy()
missing_values = missing_data_filled['Value'].values.reshape((-1, 1, 1))
predicted = autoencoder.predict(missing_values)
missing_data_filled['Value'] = predicted.reshape((-1))

703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [17]:
# 결과를 sample_submission.csv 형식으로 저장
results = pd.concat([train_data, missing_data_filled])
results.sort_index(inplace=True)  # 원래 데이터 순서대로 정렬